In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('seaborn-whitegrid')

In [ ]:
from pandas_datareader.fred import FredReader
import io
import requests

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

def format_plot(ax, recession_dates=None, xgrid=True, augment_legend=False):

    if recession_dates is not None:
        for idx, s in enumerate(recession_dates[0]):
            plt.axvspan(recession_dates[0][idx], recession_dates[1][idx], facecolor='grey', alpha=0.6, zorder=-100)

        
    if xgrid:
        ax.grid(axis='x')

    ax.spines["bottom"].set_linewidth(1.5)
    ax.spines["bottom"].set_color('k')
    ax.spines["left"].set_linewidth(1.5)
    ax.spines["left"].set_color('k')
    ax.spines["right"].set_linewidth(1.5)
    ax.spines["right"].set_color('k')
    ax.spines["top"].set_linewidth(1.5)
    ax.spines["top"].set_color('k')
    

    # add legend for recession indicator
    handles, labels = ax.get_legend_handles_labels()

    if augment_legend:
        
        if recession_dates is not None:
            handles.append(Patch(facecolor='grey',))
            labels.append("Recession indicator")
    return handles, labels  

## Get the data
For our study, we need:

* Housing price index
* household income or wages
* labor force participation
  
For context in the plots, we also want recession information.
<br>

### Data source: 

![image.png](https://fred.stlouisfed.org/images/fred-logo-2x.png)
<br>

We can use the handy FredReader class from the [pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/index.html) package!

In [ ]:
default_start_date = '1972-01-01'

## recession data

In [ ]:
recession = FredReader('USREC', start='1955-01-01').read()
recession['starts'] = (recession.USREC- recession.USREC.shift(1) ==1)
recession['ends'] = (recession.USREC- recession.USREC.shift(1) ==-1)

In [ ]:
starts = recession.index[recession['starts']==1].to_list()
ends = recession.index[recession['ends']==1].to_list()
#ends = ends[1:]

In [ ]:
starts

In [ ]:
ends

# labor force participation data

In [ ]:
#Labor Force Participation Rate (CIVPART)
labor_part = FredReader('CIVPART', start=default_start_date).read().squeeze()/100
labor_part_m = labor_part.resample('M').mean()
labor_part = labor_part.resample('Y').mean()

In [ ]:
labor_part.head()

In [ ]:
# Labor Force Participation Rate - 25-54 Yrs. (LNS11300060)
labor_part_prime = (FredReader('LNS11300060', start=default_start_date).read().squeeze()/100).resample('Y').mean()
labor_part_prime.head()

In [ ]:
#Labor Force Participation Rate - 55 Yrs. & over (LNS11324230)

labor_part_older =  (FredReader('LNS11324230', start=default_start_date).read().squeeze()/100).resample('Y').mean()
labor_part_older.head()

In [ ]:
import matplotlib.ticker as plticker



In [ ]:
ax = labor_part_m.plot(figsize = (7,4), lw=0, label='__none__')
labor_part_prime.plot(ax=ax, label='prime age', lw=2)

labor_part.plot(ax=ax, label='all civilian', lw=2)
labor_part_older.plot(ax=ax, label='older workers', lw=2)

ax.tick_params(axis='both', which='major', labelsize=12)
handles, labels = format_plot(ax, recession_dates=[starts, ends], augment_legend=True )
legend = plt.legend(handles, labels, fontsize=11, frameon=True, bbox_to_anchor=(.2,.2))
plt.setp(legend.get_title(),fontsize=12)
plt.title('Annual Labor Force Participation Trends since 1972', size=13)
plt.ylabel('Rate', size=12)
plt.xlabel('Year', size=12)


# monthly housing expense (eg mortgage)

## Realtor.com model: 

https://www.nar.realtor/sites/default/files/2025-03/hai-01-2025-housing-affordability-index-2025-03-14.pdf

The Housing Affordability Index measures whether or not a typical family earns enough income to qualify for a mortgage loan on a typical home at the national and regional levels based on the most recent price and income data.

Housing Affordability Index data are provided by NAR solely for use as a reference. No part of the data may be reproduced, stored in a retrieval system, transmitted or redistributed in any form or by any means, electronic, mechanical, photocopying, recording or otherwise, without NAR's prior written consent for those who are not members of NA

## calc basics
Home price (-20% down), with the mortgage rate, then figure monthly principal+interest payment, and compare to household income

The Realtor.com *data* is prorprotery, but nothing prevents us from re-creating the calculations. If you are a private household, trying to figure out your household budget, this is exactly what you would do. Can't see how there would be any restriction to use this in research.R.

In [ ]:
#MORTGAGE30US

mrates30 = FredReader('MORTGAGE30US', start=1972).read().squeeze()/100
mrates30 = mrates30.resample('Y').mean()
mrates30.head()

In [ ]:
ax = mrates30.plot(lw=2, figsize = (7,4))
plt.xlabel('YEAR', size=11)
plt.ylabel('Rate', size=11)
plt.title('30-year Mortgage Rates', size=13)
ax.tick_params(axis='both', which='major', labelsize=11)
#handles, labels = format_plot(ax, recession_dates=[starts, ends], augment_legend=True )

In [ ]:
sale_price = FredReader('MSPUS', start=1972).read().squeeze()/1000
sale_price.head()
sale_price_annual = sale_price.resample('Y').mean() 
sale_price_annual.plot(lw=2, figsize = (7,4))

plt.xlabel('YEAR', size=12)
plt.ylabel('Price ($thousand)', size=12)
plt.title('House Prices (Nominal dollars)', size=13)
ax.tick_params(axis='both', which='major', labelsize=11)

In [ ]:
def mort_payment(Price, i, y=30, down=0.2):
    payment = (Price*(1-down) ) * ( i/12*(1+i/12)**(12*y) )/ ((1+i/12)**(12*y) -1)
    return payment

$$ \frac{i/12 \times (1+i/12)^{12y}}{(1+i/12)^{12y}-1}$$

In [ ]:
# test
# mort_payment(360000, .07)
# 1916.0711861160453

In [ ]:
mortgage_df = pd.concat([sale_price_annual, mrates30], axis=1)
mortgage_df.dropna(inplace=True)
mortgage_df.columns = ["MedHomePrice(Th$)", "30yrMortRate"]

In [ ]:
mortgage_df["MonthlyPayment($)"] = mortgage_df.apply(lambda row: mort_payment(row["MedHomePrice(Th$)"]*1000, row["30yrMortRate"]), axis=1)

In [ ]:
mortgage_df.head()

In [ ]:
# These are in nominal dollars. the house price was in nominal;
mortgage_df.tail()

In [ ]:
mortgage_df['MonthlyPayment($)'].plot(lw=2, color='r', figsize = (7,4))

plt.xlabel('YEAR', size=11)
plt.ylabel('Payment ($)', size=11)
plt.title('Monthly Mortgage Payments (Nominal dollars)', size=13)
ax.tick_params(axis='both', which='major', labelsize=11)

# income data

In [ ]:
# MEHOINUSA646N nominal income
# only avaiable since 1984

med_income =  FredReader('MEHOINUSA646N', start=1984).read().squeeze()/12
med_income.index = pd.date_range("1984", periods=med_income.shape[0], freq ='A-DEC')
med_income.name = "MedianHHMonthlyIncome($)"
med_income.head()

In [ ]:
med_income.tail()

In [ ]:
mortgage_df = pd.concat([mortgage_df, med_income], axis=1)
mortgage_df.dropna(inplace=True)
mortgage_df.head()

In [ ]:
mortgage_df['MedianHHMonthlyIncome($)'].plot(lw=2, color='g', figsize = (6,4))

plt.xlabel('YEAR', size=11)
plt.ylabel('Income ($)', size=11)
plt.title('Median Monthly HH Income (Nominal dollars)', size=13)
ax.tick_params(axis='both', which='major', labelsize=11)

In [ ]:
# finally, create the ratio
mortgage_df['MortgageIncomeRatio'] = mortgage_df['MonthlyPayment($)']/mortgage_df['MedianHHMonthlyIncome($)']

In [ ]:
mortgage_df.head()

In [ ]:
mortgage_df.tail()

In [ ]:
ax= mortgage_df.MortgageIncomeRatio.plot(figsize=(7.5,5),lw=2, color='b')
(labor_part_prime.loc['1984':]-.6).plot(ax=ax, color='orange', lw=2)
plt.xlabel('YEAR', size=12)
plt.ylabel('Ratio', size=12)
plt.title('Ratio Mortgage Payment to Monthly HH Income', size=14)
ax.tick_params(axis='both', which='major', labelsize=11)


In [ ]:
ax = labor_part_prime.loc['1984':].plot(figsize = (7.5,5), label='all civilian', lw=2)

In [ ]:
# maybe do some kind of "phillips curve"
# HPI on y-axis or change in HPI (accelerationist)
# LFP on x axis
# like inflation/unemployment

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.regressionplots import abline_plot

In [ ]:
phil_data = pd.concat([labor_part_prime, mortgage_df['MortgageIncomeRatio'] ], axis=1)
phil_data.dropna(inplace=True)
phil_data.columns = ['LFP', 'HAI']

In [ ]:
phil_data.head()

In [ ]:
mod1 = sm.OLS(phil_data.HAI.loc['1988':], sm.add_constant(phil_data.LFP.loc['1988':]))
mod2 = sm.RLM(phil_data.HAI, sm.add_constant(phil_data.LFP))
res = mod2.fit()
res.summary()

In [ ]:
mod1.fit().summary()

$$ HAI_t = -1.047 + 1.595 LFP_t $$

In [ ]:
ax=phil_data.plot.scatter(x='LFP', y='HAI', color='k', s=30)
phil_data.loc[:'1988'].plot.scatter(ax=ax, x='LFP', y='HAI', color='r', s=25)
_=abline_plot(model_results=mod1.fit(), ax=ax, lw=2, color='r')
_=abline_plot(model_results=mod2.fit(), ax=ax, lw=2, color='b')
plt.xlabel('Labor Force Participation', size=12)
plt.ylabel('Housing Affordability', size=12)
plt.title('Housing Affordability "Phillips" Curve', size=13)
ax.tick_params(axis='both', which='major', labelsize=11)
plt.legend(fontsize=12)
#plt.text(.817, .401, s= 'HA = -1.047 + 1.595LFP', size=12, color='b', weight=2)

In [ ]:
ax=sns.scatterplot(x='LFP', y='HAI', hue=phil_data.index, data=phil_data, legend=False)